## 雙向RNN模型
- 理念
    - 一般RNN類模型只會考慮"上文"，也就是隨著順序看過的資料(如果是文字資料就是之前看過的word)，那麼雙向就是也考慮反過來，但從最後面往前去看。但要注意的是，由前往後與由後往前是獨立的，故還是有其侷限之處。
- 使用
    - 透過tf.keras.layers.Bidirectional()輕易使用!
- 參考
    - [kknews](https://kknews.cc/zh-tw/tech/g9a4q9e.html)

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [3]:
max_tokens = 20000
maxlen = 200

In [5]:
# datasets

(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_tokens)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

17465344/17464789 [==============================] - 1s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

((25000,), (25000,), (25000,), (25000,))

In [10]:
type(x_train), len(x_train[0])

(numpy.ndarray, 218)

In [11]:
# 長度不一

x_train = keras.preprocessing.sequence.pad_sequences(sequences=x_train, maxlen=maxlen, dtype='int64', padding='post')
x_test = keras.preprocessing.sequence.pad_sequences(sequences=x_test, maxlen=maxlen, dtype='int64', padding='post')

In [12]:
x_train.shape, x_test.shape

((25000, 200), (25000, 200))

In [17]:
# model

def create_model():
    inputs = keras.Input(shape=(maxlen, ), dtype='int64')           # 如果將maxlen改成None, 會變成自動識別
    x = layers.Embedding(input_dim=max_tokens, output_dim=128, input_length=maxlen)(inputs)
    x = layers.Bidirectional(layer=layers.GRU(units=32, return_sequences=True))(x)
    x = layers.Bidirectional(layer=layers.GRU(units=16, return_sequences=False))(x)
    output = layers.Dense(units=1, activation='sigmoid')(x)

    model = keras.Model(inputs, output)
    return model

bi_model = create_model()
bi_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200, 64)           31104     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 32)                7872      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,599,009
Trainable params: 2,599,009
Non-trainable params: 0
_________________________________________________________________


In [18]:
# model setup



bi_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc', 'AUC'])
bi_model.fit(
    x=x_train, y=y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.2
)

Epoch 1/5
157/157 [==============================] - 42s 226ms/step - loss: 0.6203 - acc: 0.6165 - auc: 0.6746 - val_loss: 0.3522 - val_acc: 0.8604 - val_auc: 0.9281
Epoch 2/5
157/157 [==============================] - 34s 217ms/step - loss: 0.2396 - acc: 0.9141 - auc: 0.9650 - val_loss: 0.3237 - val_acc: 0.8828 - val_auc: 0.9398
Epoch 3/5
157/157 [==============================] - 34s 216ms/step - loss: 0.1190 - acc: 0.9616 - auc: 0.9901 - val_loss: 0.3631 - val_acc: 0.8708 - val_auc: 0.9364
Epoch 4/5
157/157 [==============================] - 34s 217ms/step - loss: 0.0711 - acc: 0.9794 - auc: 0.9960 - val_loss: 0.4567 - val_acc: 0.8612 - val_auc: 0.9262
Epoch 5/5
157/157 [==============================] - 34s 217ms/step - loss: 0.0449 - acc: 0.9884 - auc: 0.9978 - val_loss: 0.4953 - val_acc: 0.8574 - val_auc: 0.9210


In [21]:
bi_model.evaluate(x_test, y_test)

782/782 [==============================] - 56s 71ms/step - loss: 0.5488 - acc: 0.8457 - auc: 0.9123


[0.5487858653068542, 0.8456799983978271, 0.912342369556427]